## Kidney disease classification 

1. Github Repository Setup
2. Project Template Creation
3. project Setup & Requirements installation
4. Logging ,Utils & Exception Module
5. Project workflows
6. All components Notebook Experiments
7. All components Modular code implementation
8. training Pipeline
9. MLflow (MLops Tool) -- For  Experiments tracking & model Registration
10. DVC(MLops Tool)  --  For Pipeline Tracking & implementation
11. Prediction pipeline & user App creation
12. Docker
13. Final CI/CD deployment on AWS